In [3]:
import json

In [4]:
file = './data/ep_ce_instances.json'
f = open(file, 'r')
data = json.load(f)['data']

In [5]:
len(data)

2531

In [6]:
gcp_data = list(filter(lambda x: x.get('provider') == 'GCP', data))

In [7]:
len(gcp_data)

190

In [13]:
ep_gcp_map = {}
for i in range(0, len(gcp_data)):
    ep_id = gcp_data[i]['id']
    variant = gcp_data[i]['variant']
    if variant.get('id'):
#         ep_gcp_map[ep_id] = variant['id']
        ep_gcp_map[ep_id] = {
            "id": variant['id'],
            "disk_type": list(map(lambda x: x.get('type'), variant['disks']))[0],
            "instanceType": variant["machineType"].split('/')[-1]
        }

In [8]:
gcp_data[0]

{'state': 'STOPPED',
 'status': 'READY',
 'volumes': [],
 'securityGroups': [{'isDefault': True,
   'status': 'RUNNING',
   'provider': 'GCP',
   '_id': '636a869d5456210011eb18f6',
   'name': 'toanntt-gcp-sg-default',
   'description': 'toanntt-gcp-sg-default',
   'owner': 'toanntt',
   'vpcId': '2938434322906074877',
   'createdBy': 'toanntt',
   'createdAt': '2022-11-08T16:41:01.830Z',
   'updatedAt': '2022-11-08T16:43:15.742Z',
   'updatedBy': None,
   'variant': {'GroupId': 'gcp-sg-636a869d5456210011eb18f6',
    'results': {'id': 'gcp-sg-636a869d5456210011eb18f6',
     'virtualmachinecount': 1,
     'virtualmachineids': ['2265529010543344118']},
    'vpcSelfLink': 'https://www.googleapis.com/compute/v1/projects/ts-engineering-337304/global/networks/default',
    'targetTags': ['bot', 'toanntt-gg-cloud']},
   'id': '636a869d5456210011eb18f6'}],
 'provider': 'GCP',
 '_id': '636a87115456210011eb1917',
 'uid': 'toanntt_11',
 'createdBy': 'toanntt',
 'createdAt': '2022-11-08T16:42:57.87

In [14]:
ep_gcp_map

{'636a87115456210011eb1917': {'id': '2265529010543344118',
  'disk_type': 'PERSISTENT',
  'instanceType': 'g1-small'},
 '636a52815456210011eadc8c': {'id': '5353556143556258917',
  'disk_type': 'PERSISTENT',
  'instanceType': 'g1-small'},
 '6367c228f6a3a80012edec42': {'id': '3922265808357194945',
  'disk_type': 'PERSISTENT',
  'instanceType': 'f1-micro'},
 '6364b95cf6a3a80012e892a1': {'id': '4036654423970744204',
  'disk_type': 'PERSISTENT',
  'instanceType': 'g1-small'},
 '6362a4d9f6a3a80012e48732': {'id': '1240637607697652238',
  'disk_type': 'PERSISTENT',
  'instanceType': 'f1-micro'},
 '63511c4ece19390019b3d94b': {'id': '6856595969022927546',
  'disk_type': 'PERSISTENT',
  'instanceType': 'g1-small'},
 '634f60f7ce19390019b047e9': {'id': '7363616063745089519',
  'disk_type': 'PERSISTENT',
  'instanceType': 'g1-small'},
 '63468d53ce193900199b7bb9': {'id': '3400834221456700298',
  'disk_type': 'PERSISTENT',
  'instanceType': 'g1-small'},
 '633a9becce1939001980ef66': {'id': '15608032217

In [33]:
df = pd.read_csv(gcp_data, header=0, sep=',')

In [58]:
df

,epId,cost,currency,date,costInUsd
0,62874b0e74b559001cc307a0,0.953786,USD,2022-07-04,0.953786
1,627b7ae49db60c001266ac77,0.839175,USD,2022-07-04,0.839175
2,63183a65ce19390019265099,0.211182,USD,2022-09-07,0.211182
3,62970ccea4414400110d0017,0.324464,USD,2022-09-07,0.324464
4,6267769ef09461001c8ec5a3,0.326841,USD,2022-09-07,0.326841
...,...,...,...,...,...
5637,6260d246b6d1ba00189e38ab,0.399614,USD,2022-09-14,0.399614
5638,6305f33ece19390019f65a97,0.834418,USD,2022-08-30,0.834418
5639,62970f07a4414400110d19e5,0.142453,USD,2022-07-27,0.142453
5640,62e15b19e55f0300115b620a,2.482112,USD,2022-08-01,2.482112


In [67]:
df['gcpId'] = df.apply(lambda row: str(ep_gcp_map.get(row.epId)), axis=1)

In [68]:
df_filtered = df.loc[df['gcpId'] != None]

In [69]:
df_filtered

,epId,cost,currency,date,costInUsd,gcpId
0,62874b0e74b559001cc307a0,0.953786,USD,2022-07-04,0.953786,6762571275092817403
1,627b7ae49db60c001266ac77,0.839175,USD,2022-07-04,0.839175,7994828870141789187
2,63183a65ce19390019265099,0.211182,USD,2022-09-07,0.211182,2084983531904533635
3,62970ccea4414400110d0017,0.324464,USD,2022-09-07,0.324464,1037551659627754032
4,6267769ef09461001c8ec5a3,0.326841,USD,2022-09-07,0.326841,1774086229609597003
...,...,...,...,...,...,...
5637,6260d246b6d1ba00189e38ab,0.399614,USD,2022-09-14,0.399614,8842805660917525667
5638,6305f33ece19390019f65a97,0.834418,USD,2022-08-30,0.834418,7440776697402117547
5639,62970f07a4414400110d19e5,0.142453,USD,2022-07-27,0.142453,2332843410177307106
5640,62e15b19e55f0300115b620a,2.482112,USD,2022-08-01,2.482112,3838479753825543664


In [66]:
df_filtered[['cost', 'date', 'costInUsd', 'gcpId']].to_csv('./data/gcp_cost.csv')